In [55]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from proj1_helpers import *
from helpers import *
from implementations import *
from cost import *
import math
from gradients import *
from preprocessing import *
from cross_validation import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [113]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'Data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

4


## Generate predictions and save ouput in csv format for submission:

In [114]:
DATA_TEST_PATH = 'Data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

4


In [150]:
minimums = []
for jet_num in range(len(ids)):
    minimums.append(ids_test[jet_num].min())
the_minimum = min(minimums)
print(the_minimum)

350000


In [151]:
tX_rename_after = tX.copy()
tX_test_rename_after = tX_test.copy()


In [152]:

tX_rename_after = remove_non_defined_columns(tX_rename_after)
tX_test_rename_after = remove_non_defined_columns(tX_test_rename_after)


In [138]:
print(np.isnan(tX_rename_after[1]))

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [250]:
# initializing prediction arrays
y_pred = []
y_total = np.zeros(len(tX_test[0]) + len(tX_test[1]) + len(tX_test[2]) + len(tX_test[3]))

#setting up parameters
degrees = [11, 12, 13, 12]
lambdas = [0.001009312, 0.001009312, 1.1212e-05, 0.0000696969]


for idx in [0]: #range(len(tX_rename_after)):
    print("beginning model ", idx+1)
    
    # initializing the prediction vector :
    y_pred_ = np.zeros(len(tX_rename_after[idx]))
    

    # extracting the values of group idx :
    train_x_jet_ = tX_rename_after[idx]
    train_y_jet_ = y[idx]
    test_x_jet_ = tX_test_rename_after[idx]
    

    # Removing additional outliers :
    train_selected_x_jet = replace_non_defined(train_x_jet_)
    test_selected_x_jet = replace_non_defined(test_x_jet_)
    
    train_selected_x_jet = train_selected_x_jet[:, np.nanstd(train_selected_x_jet, axis = 0) != 0]
    test_selected_x_jet = test_selected_x_jet[:, np.nanstd(test_selected_x_jet, axis = 0) != 0]
    
    """ i=0
    means = np.mean(train_selected_x_jet, axis = 0)
    for mean in means:
        print(mean)
        i += 1
    print(i)"""
        
    # standardize :
    tX_train_std, _, _ = standardize(train_selected_x_jet)
    tX_test_std, _, _ = standardize(test_selected_x_jet)
    
    # Polynomial feature expansion :
    """tX_train_poly = polynomial_expansion(tX_train_std, degrees[idx])
    tX_test_poly = polynomial_expansion(tX_test_std, degrees[idx])

    w_, loss_ = ridge_regression(train_y_jet_, tX_train_poly, lambdas[idx])"""
    
    _, best_param = cross_validation(train_y_jet_, tX_train_std, w_, degrees = [11],
                     lambdas=[0.001009312, 0.0011])
    
    
    print("for model ", idx+1, " best parameters are :")
    print("degree : ", best_param[0], " lambda : ", best_param[1])
    
    #accuracy_ = compute_accuracy(train_y_jet_, tX_train_poly, w_)
    #print('The accuracy of model {} is equal to {}'.format(int(idx),accuracy_))

    # Computing test accuracy : (To be changed -> à mettre dans une nouvelle boucle)
                                                
    y_pred_jet = predict_labels(w_, tX_test_poly)
    
    y_pred.append(y_pred_jet)

beginning model  1
mean for  0.001009312   0.8441243882177497
hello  0.001009312
mean for  0.0011   0.8441043708025985
for model  1  best parameters are :
degree :  11  lambda :  0.001009312


In [165]:
print(y_total[40:100])

[-1. -1. -1. -1.  1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1.
  1. -1. -1. -1.  1. -1.  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1.  1.
 -1. -1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1.  1.  1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.]


In [166]:

y_total = np.zeros(len(ids_test[0]) + len(ids_test[1]) + len(ids_test[2]) + len(ids_test[3]))

print(len(y_pred[0]) + len(y_pred[1]) + len(y_pred[2]) + len(y_pred[3]))
print(len(y_total))

for jet_num in range(len(y)):
    for j in range(len(y_pred[jet_num])):
        y_total[ids_test[jet_num][j] - the_minimum] = y_pred[jet_num][j]
        
"""
à faire : faire correspondre les résultats de prédictions correspondant à 
chaque type de particule aux index correspondants dans le y_total de taille 250k
-> peut-être en déduisant la valeur du id zéro avec la valeur de ids[jet_num][j]
"""


568238
568238


'\nà faire : faire correspondre les résultats de prédictions correspondant à \nchaque type de particule aux index correspondants dans le y_total de taille 250k\n-> peut-être en déduisant la valeur du id zéro avec la valeur de ids[jet_num][j]\n'

In [167]:
y_total.reshape(-1, 1)

array([[-1.],
       [-1.],
       [-1.],
       ...,
       [ 1.],
       [-1.],
       [-1.]])

In [168]:
ids_total = np.arange(len(y_total))
ids_total += the_minimum

print(ids_total[0])

350000


In [169]:
OUTPUT_PATH = 'Data/doto.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(ids_total, y_total, OUTPUT_PATH)